# Árvores II - Tarefa 2

### 1. Carregar as bases

Vamos carregar as bases lidas na tarefa passada. Se você salvou essas bases em arquivo texto, basta fazer a leitura com o comando ```pd.read_csv``` das seguintes bases:

- X_train
- Y_train
- X_test
- Y_test

Não se esqueça de considerar a leitura dos índices dos arquivos no ```read_csv()```!

In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [3]:
# Carregar a base X_train
X_train = pd.read_csv('X_train_processed.csv', index_col=[0, 1])

# Carregar a base y_train
y_train = pd.read_csv('input mod 17/train/y_train.txt', header=None, names=['Activity'])

# Carregar a base X_test
X_test = pd.read_csv('X_test_processed.csv', index_col=[0, 1])

# Carregar a base y_test
y_test = pd.read_csv('input mod 17/test/y_test.txt', header=None, names=['Activity'])


### 2. Divisão da base em Treino, Validação e Teste

A base já se encontra dividida em Treino e Validação. O que vamos fazer então é extrair uma base de Validação da base de Treino.

Extraia 25% da base de treino como base de validação.

In [5]:
# Dividir a base de treino em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)


### 3. Melhores 3 variáveis

Rode uma árvore com profundidade máxima igual a 4 para prever a atividade humana com todas as variáveis.
Observe a importância das variáveis e considere as 3 variáveis com maior importância para os próximos passos.
Dica: utilize o atributo ```clf.feature_importances_``` da árvore treinada.

In [6]:
# Criar o classificador da árvore de decisão
clf = DecisionTreeClassifier(max_depth=4, random_state=42)

# Treinar a árvore de decisão com todas as variáveis
clf.fit(X_train, y_train)

# Obter a importância das variáveis
importances = clf.feature_importances_

# Criar uma lista com os nomes das variáveis
feature_names = X_train.columns

# Criar um dicionário para mapear as variáveis e suas importâncias
importance_dict = dict(zip(feature_names, importances))

# Ordenar as variáveis por importância (em ordem decrescente)
sorted_importances = sorted(importance_dict.items(), key=lambda x: x[1], reverse=True)

# Obter as três variáveis com maior importância
top_3_variables = sorted_importances[:3]

# Exibir as três variáveis com maior importância
for variable, importance in top_3_variables:
    print(f"Variável: {variable}, Importância: {importance}")


Variável: 41 tGravityAcc-mean()-X, Importância: 0.28649987662995097
Variável: 84 tBodyAccJerk-std()-X, Importância: 0.2552143482681539
Variável: 42 tGravityAcc-mean()-Y, Importância: 0.16312343681954458


### 4. Construa uma árvore com as 3 melhores variáveis

Utilizando as três variáveis encontradas acima, construa uma árvore de decisão. Encontre o melhor ```ccp_alpha``` utilizando a base de validação, conforme a estrutura que vimos em aula.

In [21]:
%%time
# Selecionar as três melhores variáveis
best_variables = ['41 tGravityAcc-mean()-X', '84 tBodyAccJerk-std()-X', '42 tGravityAcc-mean()-Y']
X_train_best = X_train[best_variables]
X_val_best = X_val[best_variables]


Wall time: 1 ms


In [24]:
# Encontrar o melhor ccp_alpha utilizando a base de validação
ccp_alphas = DecisionTreeClassifier().cost_complexity_pruning_path(X_train_best, y_train)['ccp_alphas']
ccp_alphas = ccp_alphas[1:]  # Remover o primeiro valor, pois corresponde à árvore sem poda


In [25]:
# Criar uma lista para armazenar as árvores
trees = []

# Treinar uma árvore para cada ccp_alpha e calcular a acurácia na base de validação
best_accuracy = 0.0
best_ccp_alpha = None

for ccp_alpha in ccp_alphas:
    # Criar a árvore de decisão com a profundidade máxima igual a 4 e ccp_alpha específico
    clf = DecisionTreeClassifier(max_depth=4, ccp_alpha=ccp_alpha, random_state=42)
    clf.fit(X_train_best, y_train)
    trees.append(clf)
    
    # Calcular a acurácia na base de validação
    y_val_pred = clf.predict(X_val_best)
    accuracy = accuracy_score(y_val, y_val_pred)
    
    # Verificar se essa árvore é a melhor até o momento
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_ccp_alpha = ccp_alpha
        best_tree = clf

# Imprimir a melhor árvore encontrada
print("Melhor árvore com ccp_alpha =", best_ccp_alpha)
print("Acurácia na base de validação =", best_accuracy)

Melhor árvore com ccp_alpha = 9.067827348567283e-05
Acurácia na base de validação = 0.8049311094996374


### 5. Avaliação do modelo

Avalie a árvore encontrada no item anterior na base de testes.

In [27]:
# Selecionar apenas as três melhores variáveis da base de testes
X_test_best = X_test[['41 tGravityAcc-mean()-X', '84 tBodyAccJerk-std()-X', '42 tGravityAcc-mean()-Y']]

# Avaliar a árvore encontrada na base de testes
y_test_pred = best_tree.predict(X_test_best)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("Acurácia na base de testes =", test_accuracy)



Acurácia na base de testes = 0.6949440108585002
